# Visualization and Playback

### Imports

In [1]:
import mingus.core.chords as chords
import mingus.containers as mc
from mingus.midi import midi_file_out
import fluidsynth
import pandas as pd
import time
import os

In [2]:
# read from json
filepath = 'data/chords/cleaned_data.json'
full_df = pd.read_json(filepath, orient='table')
full_df.head()

,artist_name,song_name,chords,genres,popularity
0,Justin Bieber,"10,000 Hours","[G, G/B, C, G, G/B, C, G, Em, C, G, Em, C, G, ...","['canadian pop', 'pop', 'post-teen pop']",100
1,Justin Bieber,2 Much,"[F#m7, D9, F#m7, D9, F#m7, D9, E, F#m7, A/C#, ...","['canadian pop', 'pop', 'post-teen pop']",100
2,Justin Bieber,2u (feat. David Guetta),"[Em, D, C, D, Em, D, C, D, Em, D, C, Am, D, Em...","['canadian pop', 'pop', 'post-teen pop']",100
3,Justin Bieber,All Around The World,"[Em, Bm, Am, C, Em, Bm, Am, C, Em, Bm, Am, C, ...","['canadian pop', 'pop', 'post-teen pop']",100
4,Justin Bieber,All Around The World (acoustic),"[Gm, Dm, C, Gm, Dm, C, Gm, Dm, C, Gm, A#, Cdim...","['canadian pop', 'pop', 'post-teen pop']",100


## Experimenting with mingus

In [2]:
root = 'C'

In [3]:
# chords.major_triad(root)
# chords.minor_triad(root)
# chords.diminished_triad(root)
# chords.augmented_triad(root)
# chords.suspended_triad(root)

['C', 'F', 'G']

In [5]:
# chords.from_shorthand('Cm7')

['C', 'Eb', 'G', 'Bb']

In [30]:
# chords_df = full_df['chords']

# song1 = []
# for ch in chords_df[0]:
#     song1.append(chords.from_shorthand(ch))
# song1[0:10]

[['G', 'B', 'D'],
 ['B', 'G', 'B', 'D'],
 ['C', 'E', 'G'],
 ['G', 'B', 'D'],
 ['G', 'B', 'D'],
 ['B', 'G', 'B', 'D'],
 ['C', 'E', 'G'],
 ['G', 'B', 'D'],
 ['G', 'B', 'D'],
 ['E', 'G', 'B']]

## Fluidsynth

In [3]:
sound1 = "soundfont_lib/Discarded_Wurlitzer_piano.sf2"
sound2 = "soundfont_lib/YDP-GrandPiano-20160804.sf2"

In [4]:
fs = fluidsynth.Synth()
fs.start()

sfid = fs.sfload(sound2)
fs.program_select(0, sfid, 0, 0)

fs.noteon(0, 60, 30)
fs.noteon(0, 67, 30)
fs.noteon(0, 76, 30)

time.sleep(1.0)

fs.noteoff(0, 60)
fs.noteoff(0, 67)
fs.noteoff(0, 76)

time.sleep(1.0)

fs.delete()

## Playing Prog

In [5]:
chords_df = full_df['chords']

c = chords_df[0][0:10]
c

['G', 'G/B', 'C', 'G', 'G/B', 'C', 'G', 'Em', 'C', 'G']

In [6]:
t = mc.Track().from_chords(c, 1)
t

[None, [[[0.0, 1, ['G-4', 'B-4', 'D-5']]], [[0.0, 1, ['B-4', 'G-5', 'B-5', 'D-6']]], [[0.0, 1, ['C-4', 'E-4', 'G-4']]], [[0.0, 1, ['G-4', 'B-4', 'D-5']]], [[0.0, 1, ['B-4', 'G-5', 'B-5', 'D-6']]], [[0.0, 1, ['C-4', 'E-4', 'G-4']]], [[0.0, 1, ['G-4', 'B-4', 'D-5']]], [[0.0, 1, ['E-4', 'G-4', 'B-4']]], [[0.0, 1, ['C-4', 'E-4', 'G-4']]], [[0.0, 1, ['G-4', 'B-4', 'D-5']]]]]

In [19]:
# midi_files = os.listdir("data/midi")
# if midi_files == []:
#     version = 1
# else:
#     version = int(midi_files[-1][-1]) + 1
# version
version = 1

In [23]:
midi_file_out.write_Track(f'data/midi/midi_{version}.mid', t)

True

In [25]:
midipath = os.listdir("data/midi/")[-1]
midipath

'midi_1.mid'

In [28]:
fs = fluidsynth.Synth()
fs.start()

sfid = fs.sfload(sound2)
fs.program_select(0, sfid, 0, 0)

fs.play_midi_file(midipath)

0

fluidsynth: error: Couldn't open the MIDI file
